In [1]:
import msgpack
import open3d as o3d
# import trimesh
from IPython.display import display, Math
import numpy as np
import matplotlib.pyplot as plt
import time
import threading
plt.style.use('seaborn-whitegrid')
%config InlineBackend.figure_format = 'retina'

In [2]:
# https://adaickalavan.github.io/programming/udp-socket-programming-in-cpp-and-python/
# Python UDP Receiver

import socket
BUFFER_LEN = 200 #in bytes

def initUDP(IP, port):
    #Create a datagram socket
    sock = socket.socket(socket.AF_INET, # Internet
                         socket.SOCK_DGRAM) # UDP
    #Enable immediate reuse of IP address
    sock.setsockopt(socket.SOL_SOCKET,socket.SO_REUSEADDR,1)
    #Bind the socket to the port
    sock.bind((IP, port))
    #Set a timeout so the socket does not block indefinitely when trying to receive data
    sock.settimeout(1.5)

    return sock

def readUDP(sock):
    data, addr = sock.recvfrom(BUFFER_LEN)
    return data

In [3]:
ip_remote = "127.0.0.1"
port_local = 32000
port_remote = 32001
remote_address = (ip_remote,port_remote)

In [4]:
rec_sock = initUDP(ip_remote, port_local) # socket that receives udp packet
try:
    data_receive = readUDP(rec_sock)
#     print(data_receive)
    data_receive_unpacked = msgpack.unpackb(data_receive)
    print(data_receive_unpacked)
except socket.timeout:
    print("rec_socket timed out!")
rec_sock.shutdown(socket.SHUT_RDWR) # shutdown the connection
rec_sock.close()# close the connection

rec_socket timed out!


In [5]:


sen_sock = initUDP(ip_remote, port_local) # socket that send udp packet

data_send = [12,time.time(),[0,0,1,2]]
data_send_packed = msgpack.packb(data_send)

sen_sock.sendto(data_send_packed,remote_address)

sen_sock.shutdown(socket.SHUT_RDWR) # shutdown the connection
sen_sock.close()# close the connection
    


In [6]:
len(data_send_packed)

16

In [7]:
time.time()

1594806228.6372666

In [4]:
from scipy.spatial.transform import Rotation as R
def RotateFromVec(vec, angle):
    vec = np.asarray(vec, dtype=np.float64)
    vec_norm = np.linalg.norm(vec)
    if vec_norm != 1:
        vec = vec/vec_norm
    vec *= angle
    h = np.eye(4)
    h[:3, :3] = R.from_rotvec(vec).as_dcm()
    return h       

# https://www.euclideanspace.com/maths/geometry/rotations/conversions/matrixToQuaternion/
def RotMatrixToQuarternion(m):
    """
    m is a 3x3 rotation matrix or 4x4 homogenious transformation matrix
    """
    w = np.sqrt(1+m[0,0]+m[1,1]+m[2,2])/2.0
    w4 = 4*w
    x = (m[2,1]-m[1,2])/w4
    y = (m[0,2]-m[2,0])/w4
    z = (m[1,0]-m[0,1])/w4
    return np.array([w,x,y,z])

# https://www.euclideanspace.com/maths/geometry/rotations/conversions/quaternionToAngle/index.htm
def QuarternionToAxisAngle(q):
    if(q[0]>1):
        q = q/np.linalg.norm(q)
    angle = 2.0 * np.arccos(q[0])
    s = np.sqrt(1-q[0]**2)
    if(s<1e-8):
        return q[1:],angle
    else:
        return q[1:]/s,angle

In [5]:
m = RotateFromVec([0,1,1],np.pi/2.)
q = RotMatrixToQuarternion(m)
QuarternionToAxisAngle(q)

C:\Users\boxi\.conda\envs\conda_env\lib\site-packages\ipykernel_launcher.py:9: DeprecationWarning: `as_dcm` is deprecated!
as_dcm is renamed to as_matrix in scipy 1.4.0 and will be removed in scipy 1.6.0
  if __name__ == '__main__':


(array([0.        , 0.70710678, 0.70710678]), 1.5707963267948966)

In [6]:
import ipywidgets.widgets as widgets
import ipywidgets
from IPython.display import display

In [7]:
w_max = 300

In [8]:
flag_start = True
flag_reset = False

# stop button
button_stop = widgets.Button(
    description='stop',
    icon='stop',
    button_style='warning')

def OnClickButtonStop(b):
    global flag_start
    flag_start=False
button_stop.on_click(OnClickButtonStop)  

# reset button
button_reset = widgets.Button(
    description='reset',
    icon='refresh',
    button_style='info')

def OnClickButtonReset(b):
    global flag_reset
    global w0_widget
    global w1_widget
    global w2_widget
    global w3_widget
    
    flag_start=True
    w0_widget.value=0
    w1_widget.value=0
    w2_widget.value=0
    w3_widget.value=0

    
    
button_reset.on_click(OnClickButtonReset) 

w0_widget = widgets.FloatSlider(min=-w_max, max=w_max, step=1, value=0,
                                   continuous_update=True, description="front left")
w1_widget = widgets.FloatSlider(min=-w_max, max=w_max, step=1, value=0,
                                   continuous_update=True, description="back left")
w2_widget = widgets.FloatSlider(min=-w_max, max=w_max, step=1, value=0,
                                   continuous_update=True, description="front right")
w3_widget = widgets.FloatSlider(min=-w_max, max=w_max, step=1, value=0,
                                   continuous_update=True, description="back right")
caption_list = [
    widgets.Label(value='[Joint Angle]'),
    widgets.Label(value='[Orientation]'),
    widgets.Label(value='[Accelerration]'),
    widgets.Label(value='[Position]'),
]
                

ui = widgets.VBox([w0_widget, w1_widget, w2_widget, w3_widget]+caption_list)

packer = msgpack.Packer(use_single_float=False,use_bin_type =True)

def f(w0, w1, w2, w3):    
    t_now = time.time()
    data_send = (2,t_now,(w0, w1, w2, w3))
#     data_send_packed = msgpack.packb(data_send)
    data_send_packed = packer.pack(data_send) # https://msgpack-python.readthedocs.io/en/latest/api.html#msgpack.Packer
#     display(data_send_packed)
    sen_sock = initUDP(ip_remote, port_local) # socket that send udp packet
    sen_sock.sendto(data_send_packed,remote_address)
    
#     display((f"{time.time():3.3f}", (w0, w1, w2, w3)))
    
    sen_sock.shutdown(socket.SHUT_RDWR) # shutdown the connection
    sen_sock.close()# close the connection
    
out = widgets.interactive_output(f, {
    'w0': w0_widget,
    'w1': w1_widget,
    'w2': w2_widget,
    'w3': w3_widget
})
# out.layout.height = '350px'
# out.layout.width = '350px'

buttons_ui = widgets.HBox((button_stop,button_reset))

app = widgets.VBox(
    children = (ui, out,buttons_ui))

# display(app)

In [12]:
def AppControl():
    global flag_start
    flag_start=True
    
    while flag_start:
        try:
            rec_sock = initUDP(ip_remote, port_local) # socket that receives udp packet
            data_receive = readUDP(rec_sock)
            
            rec_sock.shutdown(socket.SHUT_RDWR) # shutdown the connection
            rec_sock.close()# close the connection
        #     print(data_receive)
            data_receive_unpacked = msgpack.unpackb(data_receive)
            # data checking the header should be a certain number(TODO)
            if data_receive_unpacked[0]:
                #caption_list.value=f"{data_receive_unpacked}"
                for k,caption in enumerate(caption_list):
                    caption.value = " ".join(f"{x:<+10.3f}" for x in data_receive_unpacked[2+k])
        except socket.timeout:
            print("rec_socket timed out!\r")



In [18]:
control_thread = threading.Thread(target=AppControl)
display(app)
control_thread.start()

In [43]:

button = widgets.Button(description="Click Me!")
output = widgets.Output()

display(button, output)

def on_button_clicked(b):
    with output:
        print("started")

button.on_click(on_button_clicked)

Button(description='Click Me!', style=ButtonStyle())

Output()

Button clicked.
Button clicked.
Button clicked.
Button clicked.


In [ ]:
started = True

In [47]:

# # start button
# button_start = widgets.Button(
#     description='start',
#     icon='start',
#     button_style='warning')
# button_start



Button(button_style='warning', description='stop', icon='stop', style=ButtonStyle())

In [31]:
tmp = msgpack.packb((0.00,-128.00,0.00,0.00))

In [32]:
msgpack.unpackb(tmp)

[0.0, -128.0, 0.0, 0.0]

In [35]:
packer.pack((0.00,-128.00,0.00,0.00))

b'\x94\xca\x00\x00\x00\x00\xca\xc3\x00\x00\x00\xca\x00\x00\x00\x00\xca\x00\x00\x00\x00'

In [40]:
items = [widgets.Label(str(i)) for i in range(4)]
widgets.Box(items)

Box(children=(Label(value='0'), Label(value='1'), Label(value='2'), Label(value='3')))

In [42]:
items = [widgets.Label(str(i)) for i in range(8)]
widgets.GridBox(items, layout=widgets.Layout(grid_template_columns="repeat(3, 100px)"))

GridBox(children=(Label(value='0'), Label(value='1'), Label(value='2'), Label(value='3'), Label(value='4'), La…

In [45]:
items[0].value='dfs'

In [49]:
caption_list = widgets.Label(value='The values of slider1 and slider2 are synchronized')
sliders1, slider2 = widgets.IntSlider(description='Slider 1'),\
                    widgets.IntSlider(description='Slider 2')
l = ipywidgets.link((sliders1, 'value'), (slider2, 'value'))
display(caption_list, sliders1, slider2)

Label(value='The values of slider1 and slider2 are synchronized')

IntSlider(value=0, description='Slider 1')

IntSlider(value=0, description='Slider 2')

In [51]:

a = widgets.IntSlider(description="Delayed", continuous_update=False)
b = widgets.IntText(description="Delayed", continuous_update=False)
c = widgets.IntSlider(description="Continuous", continuous_update=True)
d = widgets.IntText(description="Continuous", continuous_update=True)

ipywidgets.link((a, 'value'), (b, 'value'))
ipywidgets.link((a, 'value'), (c, 'value'))
ipywidgets.link((a, 'value'), (d, 'value'))
widgets.VBox([a,b,c,d])